In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from census import Census
import us
from us import states
import os
from shapely.geometry import Point
from shapely.wkt import loads
import requests
from IPython.display import display, HTML, Markdown as md
import xyzservices.providers as xyz
import folium
import matplotlib.pyplot as plt
from nycschools import schools, geo
from maptools import ui, address
import re


In [10]:
data = pd.read_csv("../_data/maker-incoming.csv")

# drop invalid addresses
data = data[data.parent_addr.apply(lambda x: "NY" in x)]

data.parent_addr = data.parent_addr.apply( lambda x: x.replace("SECOND STREET", "2nd ST"))
data.parent_addr = data.parent_addr.apply( lambda x: x.replace("THRID STREET", "3rd ST"))

# NYC and neighboring counties
bounds = [[40.496103, -74.272226], [41.36654, -73.423358]]
nom = Nominatim(user_agent="mixi", domain="localhost", scheme="http")
data = data.apply(lambda x: address.reverse( x, nom, addr_field="parent_addr", viewbox=bounds, multiple=True), axis=1)


data


failed to geocode: 679 VAN SINDERIN AVENUE, BROOKLYN, NY 11207
         original: 679 VAN SINDERIN AVENUE, BROOKLYN, NY 11207

failed to geocode: 82-74 89th STREET, GLENDALE, NY 11385
         original: 82-74 89 STREET, GLENDALE, NY 11385

failed to geocode: 152 NORTH ELLIOTT WALK, BROOKLYN, NY 11205
         original: 152 NORTH ELLIOTT WALK 6A, BROOKLYN, NY 11205

failed to geocode: 90-19 97th AVENUE, OZONE PARK, NY 11416
         original: 90-19 97 AVENUE, OZONE PARK, NY 11416

failed to geocode: 120 DEKRUIF PLACE, BRONX, NY 10475
         original: 120 DEKRUIF PLACE 31H, BRONX, NY 10475

failed to geocode: 21-14 45th STREET, ASTORIA, NY 11105
         original: 21-14 45 STREET, ASTORIA, NY 11105

failed to geocode: 164 BEACH 92nd STREET, FAR ROCKAWAY, NY 11693
         original: 164 BEACH 92 STREET 1, FAR ROCKAWAY, NY 11693

failed to geocode: 23-70 32nd STREET, ASTORIA, NY 11105
         original: 23-70 32 STREET 1, ASTORIA, NY 11105

failed to geocode: 225 EAST THIRD STREET, BROO

,current_school,home_language,parent_addr,geometry,lat,lon,lookup_address,full_address
0,UNITY PREP CHARTER SCHOOL (84K757),English,"679 GREENE AVENUE 4, BROOKLYN, NY 11221",POINT (-73.9411839 40.6897379),40.689738,-73.941184,"679 GREENE AVENUE, BROOKLYN, NY 11221","679 GREENE AVENUE 4\nBROOKLYN, NY 11221"
1,M.S. 297 (02M297),English,"230 MULBERRY STREET 15, NEW YORK, NY 10012",POINT (-73.9958215 40.7224536),40.722454,-73.995822,"230 MULBERRY STREET, NEW YORK, NY 10012","230 MULBERRY STREET 15\nNEW YORK, NY 10012"
2,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL (01M332),Spanish,"280 MADISON STREET 16B, NEW YORK, NY 10002",POINT (-73.9857439 40.7126465),40.712646,-73.985744,"280 MADISON STREET, NEW YORK, NY 10002","280 MADISON STREET 16B\nNEW YORK, NY 10002"
3,UNIVERSITY NEIGHBORHOOD MIDDLE SCHOOL (01M332),Spanish,"265 CHERRY STREET 16A, NEW YORK, NY 10002",POINT (-73.98927376582435 40.71088965),40.710890,-73.989274,"265 CHERRY STREET, NEW YORK, NY 10002","265 CHERRY STREET 16A\nNEW YORK, NY 10002"
4,P.S. 046 ARTHUR TAPPAN (05M046),Spanish,"99 WASHINGTON STREET #150, NEW YORK, NY 10006",POINT (-74.01411229273532 40.708272199999996),40.708272,-74.014112,"99 WASHINGTON STREET, NEW YORK, NY 10006","99 WASHINGTON STREET #150\nNEW YORK, NY 10006"
...,...,...,...,...,...,...,...,...
309,GREAT OAKS CHARTER SCHOOL (84M202),English,"165 EAST 205 STREET 1C, BRONX, NY 10458",POINT (-73.88545916314041 40.8755123),40.875512,-73.885459,"165 EAST 205th STREET, BRONX, NY 10458","165 EAST 205th STREET 1C\nBRONX, NY 10458"
310,THE URBAN ASSEMBLY UNISON SCHOOL (13K351),English,"282 LEXINGTON AVENUE B, BROOKLYN, NY 11216",POINT (-73.95040495473444 40.68752025),40.687520,-73.950405,"282 LEXINGTON AVENUE, BROOKLYN, NY 11216","282 LEXINGTON AVENUE B\nBROOKLYN, NY 11216"
311,GREAT OAKS CHARTER SCHOOL (84M202),English,"40 MADISON STREET 4G, NEW YORK, NY 10038",POINT (-73.9986889 40.7114884),40.711488,-73.998689,"40 MADISON STREET, NEW YORK, NY 10038","40 MADISON STREET 4G\nNEW YORK, NY 10038"
312,MOTT HALL II (03M862),English,"210 WEST 101 STREET 9E, NEW YORK, NY 10025",POINT (-73.9692044 40.7976293),40.797629,-73.969204,"210 WEST 101st STREET, NEW YORK, NY 10025","210 WEST 101st STREET 9E\nNEW YORK, NY 10025"


In [16]:
data = gpd.GeoDataFrame(data, geometry="geometry")
data.set_crs(epsg=4326, inplace=True)
data.explore()
data.to_file("../_data/maker-addresses.geojson", driver="GeoJSON")